# 交叉熵损失函数

In [181]:
from abc import abstractmethod, ABC
import numpy as np

np.random.seed(99)

## Foundation

### Tensor

In [182]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        topo = []
        visited = set()

        def build_topo(t):
            if t not in visited:
                visited.add(t)
                for p in t.parents:
                    build_topo(p)
                topo.append(t)

        build_topo(self)

        self.grad = np.ones_like(self.data)
        for t in reversed(topo):
            t.gradient_fn()

    def shape(self):
        return self.data.shape

    def size(self):
        return np.prod(self.data.shape[1:])

    def __str__(self):
        return str(self.data)

### Base Dataset

In [183]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size(), Tensor(self.labels).size()

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size
        return Tensor(self.features[start: end]), Tensor(self.labels[start: end])

    @abstractmethod
    def estimate(self, predictions):
        pass

### Base Layer

In [184]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, x: Tensor):
        return self.forward(x)

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def parameters(self):
        return []

    def __str__(self):
        return ''

### Base Loss Function

In [185]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### Base Optimizer

In [186]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def clear(self):
        for p in self.parameters:
            p.grad = np.zeros_like(p.data)

    @abstractmethod
    def step(self):
        pass

### Base Model

In [187]:
class Model(ABC):

    def __init__(self, layer, loss, optimizer):
        self.layer = layer
        self.loss = loss
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## Data

### MINST Dataset

In [188]:
class MINSTDataset(Dataset):

    def __init__(self, filename, batch_size=1):
        self.filename = filename
        super().__init__(batch_size)

    def load(self):
        with (np.load(self.filename, allow_pickle=True) as f):
            self.train_features, self.train_labels = self.normalize(f['x_train'], f['y_train'])
            self.test_features, self.test_labels = self.normalize(f['x_test'], f['y_test'])

    @staticmethod
    def normalize(x, y):
        inputs = x / 255
        inputs = np.expand_dims(inputs, axis=1)
        targets = np.zeros((len(y), 10))
        targets[range(len(y)), y] = 1
        return inputs, targets

    def estimate(self, predictions):
        count = (predictions.data.argmax(axis=1) == self.labels.argmax(axis=1)).sum()
        total = len(self.labels)
        return count / total

## Model

### Linear Layer

In [189]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        super().__init__()
        self.weight = Tensor(np.random.randn(out_size, in_size) * np.sqrt(2 / in_size))
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data / len(x.data)
            self.bias.grad += np.sum(p.grad, axis=0) / len(x.data)
            x.grad += p.grad @ self.weight.data / len(x.data)

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias, x}
        return p

    def parameters(self):
        return [self.weight, self.bias]

    def __str__(self):
        return f'weight: {self.weight}\nbias: {self.bias}'

### Sequential Layer

In [190]:
class Sequential(Layer):

    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def train(self):
        for l in self.layers:
            l.train()

    def eval(self):
        for l in self.layers:
            l.eval()

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

    def __str__(self):
        return '\n'.join(str(l) for l in self.layers if str(l))

### Convolutional Layer

In [191]:
class Convolution(Layer):

    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        in_size = in_channels * kernel_size * kernel_size
        self.weight = Tensor(np.random.rand(out_channels, in_channels, kernel_size, kernel_size) * np.sqrt(2 / in_size))
        self.bias = Tensor(np.zeros(out_channels))

    def forward(self, x: Tensor):
        batch, in_ch, in_h, in_w = x.data.shape
        out_h = in_h - self.kernel_size + 1
        out_w = in_w - self.kernel_size + 1
        x_padded = x.data

        patches = []
        for i in range(out_h):
            for j in range(out_w):
                patch = x_padded[:, :, i:i + self.kernel_size, j:j + self.kernel_size]
                patches.append(patch)
        patches = np.array(patches).transpose(1, 0, 2, 3, 4)
        patches_reshaped = patches.reshape(batch, out_h * out_w, -1)
        weight_reshaped = self.weight.data.reshape(self.out_channels, -1)
        output = patches_reshaped @ weight_reshaped.T + self.bias.data
        output = output.reshape(batch, out_h, out_w, self.out_channels)
        output = output.transpose(0, 3, 1, 2)
        p = Tensor(output)

        def gradient_fn():
            grad_output = p.grad.transpose(0, 2, 3, 1).reshape(batch, out_h * out_w, self.out_channels)
            weight_grad = grad_output.reshape(-1, self.out_channels).T @ patches_reshaped.reshape(-1, patches_reshaped.shape[-1])

            self.weight.grad += weight_grad.reshape(self.weight.data.shape) / batch
            self.bias.grad += np.sum(grad_output, axis=(0, 1)) / batch

            input_grad = grad_output @ weight_reshaped
            input_grad = input_grad.reshape(batch, out_h, out_w, in_ch, self.kernel_size, self.kernel_size)

            grad_input = np.zeros_like(x_padded)
            idx = 0
            for i in range(out_h):
                for j in range(out_w):
                    grad_input[:, :, i:i + self.kernel_size, j:j + self.kernel_size] += input_grad[:, i, j, :, :, :]
                    idx += 1
            x.grad += grad_input / batch

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias, x}
        return p

    def parameters(self):
        return [self.weight, self.bias]

    def __str__(self):
        return f'weight: {self.weight}\nbias: {self.bias}'

### Pooling Layer

In [192]:
class Pool(Layer):

    def __init__(self, kernel_size):
        super().__init__()
        self.kernel_size = kernel_size

    def forward(self, x: Tensor):
        batch, channels, in_h, in_w = x.data.shape
        out_h = in_h // self.kernel_size
        out_w = in_w // self.kernel_size

        output = np.zeros((batch, channels, out_h, out_w))
        mask = np.zeros_like(x.data, dtype=bool)
        for i in range(out_h):
            for j in range(out_w):
                h_start = i * self.kernel_size
                w_start = j * self.kernel_size
                region = x.data[:, :, h_start:h_start + self.kernel_size, w_start:w_start + self.kernel_size]
                max_val = np.max(region, axis=(2, 3), keepdims=True)
                output[:, :, i, j] = max_val.squeeze()
                region_mask = (region == max_val)
                mask[:, :, h_start:h_start + self.kernel_size, w_start:w_start + self.kernel_size] |= region_mask
        p = Tensor(output)

        def gradient_fn():
            grad_input = np.zeros_like(x.data)
            for i in range(out_h):
                for j in range(out_w):
                    h_start = i * self.kernel_size
                    w_start = j * self.kernel_size
                    region_mask = mask[:, :, h_start:h_start + self.kernel_size, w_start:w_start + self.kernel_size]
                    grad_region = p.grad[:, :, i:i + 1, j:j + 1]
                    grad_input[:, :, h_start:h_start + self.kernel_size, w_start:w_start + self.kernel_size] += region_mask * grad_region
            x.grad += grad_input

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### Flatten Layer

In [193]:
class Flatten(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.array(x.data.reshape(x.data.shape[0], -1)))

        def gradient_fn():
            x.grad += p.grad.reshape(x.data.shape)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### Dropout Layer

In [194]:
class Dropout(Layer):

    def __init__(self, dropout_rate=0.3):
        super().__init__()
        self.dropout_rate = dropout_rate

    def forward(self, x: Tensor):
        if not self.training:
            return x

        mask = np.random.random(x.data.shape) > self.dropout_rate
        p = Tensor(x.data * mask)

        def gradient_fn():
            x.grad += p.grad * mask

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### ReLU Activation Function

In [195]:
class ReLU(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.maximum(0, x.data))

        def gradient_fn():
            x.grad += (p.data > 0) * p.grad

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### Tanh Activation Function

In [196]:
class Tanh(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.tanh(x.data))

        def gradient_fn():
            x.grad += p.grad * (1 - p.data ** 2)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### Sigmoid Activation Function

In [197]:
class Sigmoid(Layer):

    def __init__(self, clip_range=(-100, 100)):
        super().__init__()
        self.clip_range = clip_range

    def forward(self, x: Tensor):
        z = np.clip(x.data, self.clip_range[0], self.clip_range[1])
        p = Tensor(1 / (1 + np.exp(-z)))

        def gradient_fn():
            x.grad += p.grad * p.data * (1 - p.data)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### Softmax Activation Function

In [198]:
class Softmax(Layer):

    def __init__(self, axis=-1):
        super().__init__()
        self.axis = axis

    def forward(self, x: Tensor):
        exp = np.exp(x.data - np.max(x.data, axis=self.axis, keepdims=True))
        p = Tensor(exp / np.sum(exp, axis=self.axis, keepdims=True))

        def gradient_fn():
            grad = np.sum(p.data * p.grad, axis=self.axis, keepdims=True)
            x.grad += p.data * (p.grad - grad)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### MSE Loss Function

In [199]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### Cross Entropy Loss Function

In [200]:
class CELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        exp = np.exp(p.data - np.max(p.data, axis=-1, keepdims=True))
        softmax = exp / np.sum(exp, axis=-1, keepdims=True)

        log = np.log(np.clip(softmax, 1e-10, 1))
        ce = Tensor(0 - np.sum(y.data * log) / len(p.data))

        def gradient_fn():
            p.grad += (softmax - y.data) / len(p.data)

        ce.gradient_fn = gradient_fn
        ce.parents = {p}
        return ce

### Binary Cross Entropy Loss Function

In [201]:
class BCELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        clipped = np.clip(p.data, 1e-7, 1 - 1e-7)
        bce = Tensor(-np.mean(y.data * np.log(clipped)
                              + (1 - y.data) * np.log(1 - clipped)))

        def gradient_fn():
            p.grad += (clipped - y.data) / (clipped * (1 - clipped) * len(p.data))

        bce.gradient_fn = gradient_fn
        bce.parents = {p}
        return bce

### SGD Optimizer

In [202]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### Neural Network Model

In [203]:
class NNModel(Model):

    def train(self, dataset, epochs):
        self.layer.train()
        dataset.train()

        for epoch in range(epochs):
            for i in range(len(dataset)):
                features, labels = dataset[i]

                predictions = self.layer(features)
                error = self.loss(predictions, labels)

                self.optimizer.clear()
                error.backward()
                self.optimizer.step()

    def test(self, dataset):
        self.layer.eval()
        dataset.eval()

        features, labels = dataset.items()
        return self.layer(features)

## Configuration

### Learning Rate

In [204]:
LEARNING_RATE = 0.01

### Batch

In [205]:
BATCH_SIZE = 2

### Kernel

In [206]:
KERNEL_SIZE = 3

### Pool

In [207]:
POOL_SIZE = 2

### Epoch

In [208]:
EPOCHS = 20

## Training

### Iteration Training

In [209]:
dataset = MINSTDataset('tinymnist.npz', BATCH_SIZE)
feature, label = dataset[0]
_, channels, rows, columns = feature.shape()
conv_rows = (rows - KERNEL_SIZE + 1) // POOL_SIZE
conv_columns = (columns - KERNEL_SIZE + 1) // POOL_SIZE
layer = Sequential([Convolution(channels, 16, KERNEL_SIZE),
                    Pool(POOL_SIZE),
                    Flatten(),
                    Dropout(),
                    Linear(conv_rows * conv_columns * 16, 64),
                    ReLU(),
                    Linear(64, dataset.shape()[1])])
loss = CELoss()
optimizer = SGDOptimizer(layer.parameters(), lr=LEARNING_RATE)

model = NNModel(layer, loss, optimizer)
model.train(dataset, EPOCHS)

## Testing

### Estimating

In [210]:
predictions = model.test(dataset)

print(f'Accuracy: {dataset.estimate(predictions)}')

Accuracy: 0.935
